In [1]:
import pandas as pd
import numpy as np
import re

# Funções utilizadas

In [2]:
# @title Criando o dataframe
# Inicializando os vetores que serão utilizados para receber os times nas respectivas fases
campeao = []
vice = []
finalistas = []
participantes = []
# Colunas do Dataframe
colunas_df = ['Time', 'MUN_A_1', 'MUN_A_2', 'MUN_A_3', 'MUN_A_4', 'MUN_A_8', 'MUN_B_1', 'MUN_B_2', 'MUN_C_1', 'MUN_C_3', 'MUN_P']
df_copa = pd.DataFrame(columns=colunas_df)
df_copa

,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P


Dicionário de colunas



| Nome | Descrição |
|------|-----------|
| MUN_x_1  | Vencedor da edição |
| MUN_x_2  | Vice da Edição |
| MUN_x_3  | 3o Lugar |
| MUN_x_4  | 4o Lugar |
| MUN_x_8  | Entre o Top 8 |
| MUN_P    | Número de participações |

In [3]:
# @title Função get Table Malformatada
def get_table_unformatted(dataframe):
  # Remover colunas multinivel e colunas que não serão utilizadas
  dataframe = dataframe.drop(dataframe.columns[-2:], axis=1)
  dataframe.columns = dataframe.columns.droplevel(0)
  dataframe.columns = dataframe.columns.get_level_values(0)
  dataframe['Resultado'] = ""
  res = "Campeão"
  indices = []
  for index, row in dataframe.iterrows():
      time = row['Times']
      if 'campeão' in time or 'Eliminados' in time:
        res = time
        indices.append(index)
      else:
        dataframe.at[index, 'Resultado'] = res
  dataframe.drop(indices, inplace=True)
  return dataframe

In [4]:
# @title Função limpar nomes
# Usando regex para limpar o '(x)' no final do nome do time
def clear_text(text):
    text = pd.Series(text).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
    # Remove espaços em branco no início e no final
    return text.str.strip()

In [5]:
# @title Função get Bracket
# Esta função pega o chaveamento da copa na Wikipedia
def get_bracket(final, bracket):
  top_4 = []
  top_qf = []
  top_of = []
  top_f1 = []
  bracket = bracket.loc[2:, :]
  participantes = final
  print(f'Finalistas: {final}')

  # A partir do final do Chaveamento anda as colunas até a fase inicial para recuperar os times
  for i in range(len(bracket.columns)-1, 0, -1):

      # Pegar os times sem repetição na coluna desejada
      times = bracket.loc[:, i].dropna().unique()
      # Retira os possíveis () e []
      times = pd.Series(times).replace(r'\[.*?\]', '', regex=True).replace(r'\(.*?\)', '', regex=True)
      # Define o tipo como String
      times = times.astype(str)
      # Garante que o conteúdo não contém apenas números
      times = times[~times.str.match(r'^[0-9\s]+$')]
      times = times.to_list()
      # Limpa o conteúdo
      times = [team.strip() for team in times if isinstance(team, str) and re.match(r'^(?!\d+$)[A-Za-zÀ-ÿ0-9\s\-]+$', team)]

      # Se a quantidade de times retornada for igual a 4, quer dizer que a coluna é a semifinal
      if (len(times) == 4) & (len(top_4) == 0):
          # Recupera os times que ainda não estão na lista 'participantes'
          top_4 = [team for team in times if team not in participantes]
          print(f'Semis:  {top_4}')
          # Adiciona os times da semifinal à lista 'participantes'
          participantes = set(participantes).union(top_4)

      elif (len(times) == 8) & (len(top_qf) == 0): # 8 times = coluna Quartas de final
          top_qf = [team for team in times if team not in participantes]
          print(f'Quartas: {top_qf}')
          participantes = set(participantes).union(top_qf)

      elif (len(times) == 16) & (len(top_of) == 0): # 16 times = coluna Oitavas de final
          top_of = [team for team in times if team not in participantes]
          print(f'Oitavas: {top_of}')
          participantes = set(participantes).union(top_of)

      elif (len(times) > 16) & (len(times) <= 32) & (len(top_f1) == 0): # Entre 16 e 32 times = coluna -1F
          top_f1 = [team for team in times if team not in participantes]
          print(f'1ª Fase: {top_f1}')
          participantes = set(participantes).union(top_f1)

  return participantes, top_4, top_qf, top_of, top_f1

In [6]:
# @title Função get Table
def get_table(table, ano):
  tabela = table.rename(columns={table.columns[0]: 'Pos', table.columns[1]: 'Time'})
  #dftemp['Pos'] = dftemp['Pos'].astype(int)
  campeao = []
  vice = []
  top_4 = []
  top_3 = []
  top_8 = []
  if (ano == 2024):
    print(table)
  for i, row in tabela.iterrows():
    pos = float(row["Pos"])
    time = clear_text(row["Time"]).to_list()

    if pos == 1:
      campeao = time
    elif pos == 2:
      vice = time
    elif (pos == 3):
      top_3 = top_3 + time
    elif (pos == 4):
      top_4 = top_4 + time
    elif (pos <= 8):
      top_8 = top_8 + time

  print(f'Participantes: {len(tabela)}')
  print(f'Campeão: {campeao}')
  print(f'Vice: {vice}')
  print(f'3º Lugar: {top_3}')
  print(f'4º Lugar: {top_4}')
  print(f'5º ao 8º: {top_8}')

  participantes = campeao + vice + top_3 + top_4 + top_8

  return participantes, top_4, top_8

In [7]:
# @title Função preencher tabela
def preencher_tabela(times_ed, df):
  times_tabela = df['Time'].to_list()
  # Itera sobre os participantes da edição
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df.loc[len(df)] = [team] + [0] * (len(df.columns) - 1)
          # df.loc[len(df)]

      if pos == 1:
          df.loc[df['Time'] == team, 'LIG_A_1'] += 1
      elif pos == 2:
          df.loc[df['Time'] == team, 'LIG_A_2'] += 1
      elif (pos <= 4):
          df.loc[df['Time'] == team, 'LIG_A_4'] += 1
      elif (pos <= 8):
          df.loc[df['Time'] == team, 'LIG_A_8'] += 1
      elif (pos <= 12):
          df.loc[df['Time'] == team, 'LIG_A_12'] += 1
      elif team in reb:
          df.loc[df['Time'] == team, 'LIG_A_R'] += 1
      elif team in top_of:
          df.loc[df['Time'] == team, 'LIG_A_NR'] += 1
      elif (pos <= 16):
          df.loc[df['Time'] == team, 'LIG_A_16'] += 1
      elif (pos <= 24):
          df.loc[df['Time'] == team, 'LIG_A_24'] += 1
      elif (pos <= 32):
          df.loc[df['Time'] == team, 'LIG_A_32'] += 1
      elif (pos <= 64):
          df.loc[df['Time'] == team, 'LIG_A_64'] += 1
      elif (pos <= 100):
          df.loc[df['Time'] == team, 'LIG_A_100'] += 1

      if team in ultimos:
          df.loc[df['Time'] == team, 'LIG_A_ULT'] += 1

      # Adiciona a participação do time na edição
      df.loc[df['Time'] == team, 'LIG_A_P'] += 1

  return df

# 1951 - 1º Mundial

In [20]:
df = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_clubes_campe%C3%B5es_mundiais_de_futebol')
df = df[0]
df.columns = df.columns.droplevel(0)
df

,Ano,Campeão,Placar(es),Vice-campeão,Estádio e país
0,1960 Detalhes,Real Madrid,0 – 0 5 – 1,Peñarol,"Centenário, Uruguai (1º jogo) Santiago Bernabé..."
1,1961 Detalhes,Peñarol,0 – 1 5 – 0 2 – 1,Benfica,"Estádio da Luz, Portugal (1º jogo) Centenário ..."
2,1962 Detalhes,Santos,3 – 2 5 – 2,Benfica,"Maracanã, Brasil (1º jogo) Estádio da Luz, Por..."
3,1963 Detalhes,Santos,2 – 4 4 – 2 1 – 0,Milan,"San Siro, Itália (1º jogo) Maracanã, Brasil (2..."
4,1964 Detalhes,Internazionale,1 – 0 2 – 0 1 – 0 (pro),Independiente,"La Doble Visera, Argentina (1º jogo) San Siro,..."
...,...,...,...,...,...
74,2025 Detalhes,A disputar,A disputar,A disputar,NaN
75,Mundial de Clubes FIFA (2025 – presente),Mundial de Clubes FIFA (2025 – presente),Mundial de Clubes FIFA (2025 – presente),Mundial de Clubes FIFA (2025 – presente),Mundial de Clubes FIFA (2025 – presente)
76,Ano,Final,Final,Final,País sede
77,Ano,Campeão,Placar,Vice-campeão,País sede


In [21]:
df_1 = df.iloc[0, :4]
df_1['Ano'] = df_1['Ano'].replace(" Detalhes", "")

ano = df_1['Ano']
campeao = df_1['Campeão']
vice = df_1['Vice-campeão']
terceiro = "Vasco da Gama"

print(f'Mundial de {ano}')
print(f'Campeão: {campeao}')
print(f'Vice: {vice}')
print(f'3º Lugar: {terceiro}')

df_copa.loc[len(df_copa)] = [campeao] + [0] * (len(df_copa.columns) - 1)
df_copa.loc[df_copa['Time'] == campeao, 'MUN_C_1'] += 1
df_copa.loc[df_copa['Time'] == campeao, 'MUN_P'] += 1
df_copa.loc[len(df_copa)] = [terceiro] + [0] * (len(df_copa.columns) - 1)
df_copa.loc[df_copa['Time'] == terceiro, 'MUN_C_3'] += 1
df_copa.loc[df_copa['Time'] == terceiro, 'MUN_P'] += 1
df_copa

Mundial de 1960
Campeão: Real Madrid
Vice: Peñarol
3º Lugar: Vasco da Gama


,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P
0,Real Madrid,0,0,0,0,0,0,0,1,0,1
1,Vasco da Gama,0,0,0,0,0,0,0,0,1,1


# 1960-2004: Intercontinental

In [24]:
df_2 = df.iloc[5:45, :4]
df_2.drop(20, inplace=True)
df_2.drop(23, inplace=True)
df_2.reset_index(drop=True)
df_2['Ano'] = df_2['Ano'].str.replace(" Detalhes", "")
df_2

,Ano,Campeão,Placar(es),Vice-campeão
5,1965,Internazionale,3 – 0 0 – 0,Independiente
6,1966,Peñarol,3 – 0 0 – 0,Real Madrid
7,1967,Racing,0 – 1 2 – 1 1 – 0,Celtic
8,1968,Estudiantes,1 – 0 1 – 1,Manchester United
9,1969,Milan,3 – 0 1 – 2,Estudiantes
10,1970,Feyenoord,1 – 0 1 – 1,Estudiantes
11,1971,Nacional,1 – 1 2 – 1,Panathinaikos
12,1972,Ajax,1 – 1 3 – 0,Independiente
13,1973,Independiente,1 – 0,Juventus
14,1974,Atlético de Madrid,0 – 1 2 – 0,Independiente


In [25]:
participantes = []
for i in range(len(df_2)):
  ano = int(df_2.iloc[i]['Ano'])
  campeao = df_2.iloc[i]['Campeão']
  vice = df_2.iloc[i]['Vice-campeão']
  print(f'\nMundial de {ano}')
  print(f'Campeão: {campeao}')
  print(f'Vice: {vice}')

  times_ed = [campeao, vice]
  times_tabela = df_copa['Time'].to_list()
  # Itera sobre os participantes da edição
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df_copa.loc[len(df_copa)] = [team] + [0] * (len(df_copa.columns) - 1)
          # df_copa.loc[len(df_copa)]
      if (ano >= 1960) & (ano <= 2004):
        if pos == 1:
            df_copa.loc[df_copa['Time'] == team, 'MUN_B_1'] += 1
        elif pos == 2:
            df_copa.loc[df_copa['Time'] == team, 'MUN_B_2'] += 1

      # Adiciona a participação do time na edição
      df_copa.loc[df_copa['Time'] == team, 'MUN_P'] += 1



Mundial de 1965
Campeão: Internazionale
Vice: Independiente

Mundial de 1966
Campeão: Peñarol
Vice: Real Madrid

Mundial de 1967
Campeão: Racing
Vice: Celtic

Mundial de 1968
Campeão: Estudiantes
Vice: Manchester United

Mundial de 1969
Campeão: Milan
Vice: Estudiantes

Mundial de 1970
Campeão: Feyenoord
Vice: Estudiantes

Mundial de 1971
Campeão: Nacional
Vice: Panathinaikos

Mundial de 1972
Campeão: Ajax
Vice: Independiente

Mundial de 1973
Campeão: Independiente
Vice: Juventus

Mundial de 1974
Campeão: Atlético de Madrid
Vice: Independiente

Mundial de 1975
Campeão: Não realizado Bayern de Munique x Independiente Ambos os times não acertaram datas para jogar.
Vice: Não realizado Bayern de Munique x Independiente Ambos os times não acertaram datas para jogar.

Mundial de 1976
Campeão: Bayern de Munique
Vice: Cruzeiro

Mundial de 1977
Campeão: Boca Juniors
Vice: Borussia Mönchengladbach

Mundial de 1978
Campeão: Não realizado Liverpool x Boca Juniors Ambos os times desistiram de joga

In [26]:
df_copa

,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P
0,Real Madrid,0,0,0,0,0,4,4,1,0,9
1,Vasco da Gama,0,0,0,0,0,0,2,0,1,3
2,Internazionale,0,0,0,0,0,2,0,0,0,2
3,Independiente,0,0,0,0,0,4,6,0,0,10
4,Peñarol,0,0,0,0,0,4,2,0,0,6
5,Racing,0,0,0,0,0,2,0,0,0,2
6,Celtic,0,0,0,0,0,0,2,0,0,2
7,Estudiantes,0,0,0,0,0,2,4,0,0,6
8,Manchester United,0,0,0,0,0,2,2,0,0,4
9,Milan,0,0,0,0,0,6,6,0,0,12


# Mundial da FIFA 2000/2005-2024

## 2000-2021 Classificação Final

In [27]:
df = pd.read_html('https://pt.wikipedia.org/w/index.php?title=Copa_Intercontinental_da_FIFA_de_2024&oldid=69888191')
df[21]

,0,1
0,Outros projetos Wikimedia também contêm materi...,Outros projetos Wikimedia também contêm materi...
1,NaN,Base de dados no Wikidata


In [28]:
# 2000, 18: t=28
# 2005: t=15
# 2006, 12-13, 15: t=17
# 2007-08: t=16
# 2009-10: t=18
# 2011: t=19
# 2014: t=20
# 2016, 19, 21: t=26
# 2017, 20: t=25
# 2022: t=24
# 2023: t=22
# 2024: t=20
for i in range(2000, 2023+1):
  if 2001 <= i <= 2004:
        continue
  ano = str(i)
  #2011-2023: link diferente
  if i >= 2011:
    df = pd.read_html('https://pt.wikipedia.org/wiki/Copa_do_Mundo_de_Clubes_da_FIFA_de_' + ano)
  elif i == 2024:
    df = pd.read_html('https://pt.wikipedia.org/w/index.php?title=Copa_Intercontinental_da_FIFA_de_2024&oldid=69888191')
  else:
    df = pd.read_html('https://pt.wikipedia.org/wiki/Campeonato_Mundial_de_Clubes_da_FIFA_de_' + ano)
  t = 0
  print(f'\nMundial de {i}:')
  if (i == 2000) | (i == 2018):
    print(ano)
    t = 28
  elif (i == 2005):
    t = 15
  elif (i == 2006) | (i == 2012) | (i == 2013) | (i == 2015):
    t = 17
  elif (i == 2007) | (i == 2008):
    t = 16
  elif (i == 2009) | (i == 2010):
    t = 18
  elif (i == 2011):
    t = 19
  elif (i == 2014) | (i == 2024):

    t = 20
    print("2024")
    print(f"t={t}")
    print(df[t])
  elif (i == 2016) | (i == 2019) | (i == 2021):
    t = 26
  elif (i == 2017) | (i == 2020):
    t = 25
  elif (i == 2022):
    t = 24
  elif (i == 2023):
    t = 22

  times_tabela = df_copa['Time'].to_list()
  # Itera sobre os participantes da edição
  times_ed, _, _ = get_table(df[t], int(ano))
  for pos, team in enumerate(times_ed):
      pos += 1
      # Insere o time caso seja a primeira participação
      if team not in times_tabela:
          print(f'Novo participante: {team}')
          df_copa.loc[len(df_copa)] = [team] + [0] * (len(df_copa.columns) - 1)
          # df_copa.loc[len(df_copa)]

      if pos == 1:
          df_copa.loc[df_copa['Time'] == team, 'MUN_A_1'] += 1
      elif pos == 2:
          df_copa.loc[df_copa['Time'] == team, 'MUN_A_2'] += 1
      elif (pos == 3):
          df_copa.loc[df_copa['Time'] == team, 'MUN_A_3'] += 1
      elif (pos == 4):
          df_copa.loc[df_copa['Time'] == team, 'MUN_A_4'] += 1
      elif (pos >= 5) & (pos <= 8):
          df_copa.loc[df_copa['Time'] == team, 'MUN_A_8'] += 1

      # Adiciona a participação do time na edição
      df_copa.loc[df_copa['Time'] == team, 'MUN_P'] += 1


Mundial de 2000:
2000
Participantes: 8
Campeão: ['Corinthians']
Vice: ['Vasco da Gama']
3º Lugar: ['Necaxa']
4º Lugar: ['Real Madrid']
5º ao 8º: ['Manchester United', 'Al-Nassr', 'Raja Casablanca', 'South Melbourne']
Novo participante: Corinthians
Novo participante: Necaxa
Novo participante: Al-Nassr
Novo participante: Raja Casablanca
Novo participante: South Melbourne

Mundial de 2005:
Participantes: 6
Campeão: ['São Paulo']
Vice: ['Liverpool']
3º Lugar: ['Deportivo Saprissa']
4º Lugar: ['Al-Ittihad']
5º ao 8º: ['Sydney FC', 'Al-Ahly']
Novo participante: Deportivo Saprissa
Novo participante: Al-Ittihad
Novo participante: Sydney FC
Novo participante: Al-Ahly

Mundial de 2006:
Participantes: 6
Campeão: ['Internacional']
Vice: ['Barcelona']
3º Lugar: ['Al-Ahly']
4º Lugar: ['América']
5º ao 8º: ['Jeonbuk Motors', 'Auckland City']
Novo participante: Internacional
Novo participante: América
Novo participante: Jeonbuk Motors
Novo participante: Auckland City

Mundial de 2007:
Participantes: 

# Intercontinental da FIFA 2024-

In [29]:
campeao = "Botafogo"
df_copa.loc[len(df_copa)] = [campeao] + [0] * (len(df_copa.columns) - 1)
df_copa.loc[df_copa['Time'] == campeao, 'MUN_A_8'] += 1
df_copa.loc[df_copa['Time'] == campeao, 'MUN_P'] += 1
df_copa

,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P
0,Real Madrid,5,0,0,1,0,4,4,1,0,15
1,Vasco da Gama,0,1,0,0,0,0,2,0,1,4
2,Internazionale,1,0,0,0,0,2,0,0,0,3
3,Independiente,0,0,0,0,0,4,6,0,0,10
4,Peñarol,0,0,0,0,0,4,2,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...
100,Seattle Sounders,0,0,0,0,1,0,0,0,0,1
101,Manchester City,1,0,0,0,0,0,0,0,0,1
102,Fluminense,0,1,0,0,0,0,0,0,0,1
103,León,0,0,0,0,1,0,0,0,0,1


# Recuperar participantes

In [30]:
df_part = pd.read_html('https://pt.wikipedia.org/wiki/Participa%C3%A7%C3%B5es_de_clubes_brasileiros_de_futebol_em_competi%C3%A7%C3%B5es_internacionais#Copa_Sul-Americana')
part_mun_a = df_part[1]
part_mun_a_times = part_mun_a['Clube'].to_list()
part_mun_a_times

['Palmeiras', 'Flamengo', 'Fluminense', 'Botafogo']

In [31]:
df_part = pd.read_html('https://pt.wikipedia.org/wiki/Participa%C3%A7%C3%B5es_de_clubes_brasileiros_de_futebol_em_competi%C3%A7%C3%B5es_internacionais#Copa_Sul-Americana')
part_mun_b = df_part[5]
part_mun_b_times = part_mun_b['Clube'].to_list()
part_mun_b_times

['Corinthians',
 'Internacional',
 'Flamengo',
 'Palmeiras',
 'São Paulo',
 'Fluminense',
 'Grêmio',
 'Santos',
 'Vasco da Gama',
 'Atlético Mineiro']

In [32]:
df_part = pd.read_html('https://pt.wikipedia.org/wiki/Participa%C3%A7%C3%B5es_de_clubes_brasileiros_de_futebol_em_competi%C3%A7%C3%B5es_internacionais#Copa_Sul-Americana')
part_mun_c = df_part[10]
part_mun_c_times = part_mun_c['Clube'].to_list()
part_mun_c_times

['Santos',
 'Cruzeiro',
 'Grêmio',
 'São Paulo',
 'Flamengo',
 'Vasco da Gama',
 'Palmeiras']

In [33]:
lista_part_br = list(set(part_mun_a_times + part_mun_b_times + part_mun_c_times))
lista_part_br.sort()

In [34]:
df_copa = df_copa[df_copa['Time'].isin(lista_part_br)].reset_index(drop=True)
df_copa

,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P
0,Vasco da Gama,0,1,0,0,0,0,2,0,1,4
1,Cruzeiro,0,0,0,0,0,0,4,0,0,4
2,Flamengo,0,1,1,0,0,2,0,0,0,4
3,São Paulo,1,0,0,0,0,4,0,0,0,5
4,Grêmio,0,1,0,0,0,0,2,0,0,3
5,Palmeiras,0,1,0,1,0,0,2,0,0,4
6,Corinthians,2,0,0,0,0,0,0,0,0,2
7,Internacional,1,0,1,0,0,0,0,0,0,2
8,Santos,0,1,0,0,0,0,0,0,0,1
9,Atlético Mineiro,0,0,1,0,0,0,0,0,0,1


# Pontuação

| Pct %   | Resultado | Copa Rio 1951     | Intercontinental | Mundial da FIFA     |
|-|-|-|-|-|
|         |  | 14400 | 15120 | 15876 |
|
| 100.00% | Campeão        |
|  33.33% | Vice           |
|
|  16.67% | 3o Lugar       | 	       | 	X  |
|  11.11% | 4o Lugar       | 	       |  X  |
|   5.56% | Top 8          | 	       | 	X  |








In [35]:
df_rank = df_copa.copy()

In [36]:
# Pontuação em relação apenas ao Campeonato Brasileiro
mun_c = 40
mun_b = 85
mun_a = 100
df_rank['PTS'] = round(df_rank['MUN_A_1']*mun_a +
                  df_rank['MUN_A_2']*mun_a/3 +
                  df_rank['MUN_A_3']*mun_a/6 +
                  df_rank['MUN_A_4']*mun_a/9 +
                  df_rank['MUN_A_8']*mun_a/18 +
                  df_rank['MUN_B_1']*mun_b +
                  df_rank['MUN_B_2']*mun_b/3 +
                  df_rank['MUN_C_1']*mun_c +
                  df_rank['MUN_C_3']*mun_c/6
, 2)

In [37]:
df_rank = df_rank.sort_values(by=['PTS', 'Time'], ascending=[False, True]).reset_index(drop=True)
df_rank.index += 1

# Ranking Geral

In [38]:
df_rank

,Time,MUN_A_1,MUN_A_2,MUN_A_3,MUN_A_4,MUN_A_8,MUN_B_1,MUN_B_2,MUN_C_1,MUN_C_3,MUN_P,PTS
1,São Paulo,1,0,0,0,0,4,0,0,0,5,440.00
2,Flamengo,0,1,1,0,0,2,0,0,0,4,220.00
3,Corinthians,2,0,0,0,0,0,0,0,0,2,200.00
4,Internacional,1,0,1,0,0,0,0,0,0,2,116.67
5,Cruzeiro,0,0,0,0,0,0,4,0,0,4,113.33
6,Palmeiras,0,1,0,1,0,0,2,0,0,4,101.11
7,Vasco da Gama,0,1,0,0,0,0,2,0,1,4,96.67
8,Grêmio,0,1,0,0,0,0,2,0,0,3,90.00
9,Fluminense,0,1,0,0,0,0,0,0,0,1,33.33
10,Santos,0,1,0,0,0,0,0,0,0,1,33.33


In [39]:
# Exportar para Excel
df_rank.to_excel('Mundial Rank.xlsx')